# KundenKlasse

In [1]:
import datetime
import random
#from Bankkontodatei import Bankkontoklasse as Bankkonto
#from Bankdatei import Bankklasse as Bank

class Kundenklasse:
    def __init__(self, vorname, nachname, geburtsdatum, adresse, telefonnummer, email, geschlecht):
        self.vorname = vorname
        self.nachname = nachname
        self.geburtsdatum = datetime.datetime.strptime(geburtsdatum, "%d.%m.%Y").date()
        self.alter = self.alter_berechnen()
        self.kundennummer = self.kundennummer_generator()
        self.adresse = adresse
        self.telefonnummer = telefonnummer
        self.email = email
        self.name = vorname + " " + nachname
        self.geschlecht = geschlecht

    def kundennummer_generator(self):
        kundennummer = "".join([str(random.randint(0, 9)) for _ in range(10)])
        print(f"Generierte Kundennummer: {kundennummer}")
        return kundennummer

    def alter_berechnen(self):
        heute = datetime.date.today()
        alter = heute.year - self.geburtsdatum.year
        # Prüfe, ob der Geburtstag in diesem Jahr schon war
        if (heute.month, heute.day) < (self.geburtsdatum.month, self.geburtsdatum.day):
            alter -= 1
        return alter

    def Kundendetails(self):
        print(f"Name: {self.name}\nAlter {self.alter}\nAdresse: {self.adresse}\nTelefonnummer: {self.telefonnummer}\nE-Mail: {self.email}\nGeschlecht: {self.geschlecht}")


# kunde1 = Kundenklasse("Anna", "Schmidt", "15.05.1990", "Beispielweg 5, 12345 Berlin", "0171-1234567", "anna.schmidt@email.com", "weiblich")


# BankkontoKlasse

In [ ]:
class Bankkontoklasse:
    """
    Repräsentiert ein Bankkonto mit grundlegenden Funktionen wie Einzahlen, Abheben und Transaktionshistorie.

    Attributes:
        kontonummer (str): Die Kontonummer des Kontos.
        inhaber (Kundenklasse): Der Kontoinhaber.
        bank (Bankklasse): Die zugehörige Bank.
        iban (str): Die IBAN des Kontos.
        pin (str): Die PIN für das Konto.
        saldo (float): Der aktuelle Kontostand.
        dispo (float): Der Dispositionskredit.
        transaktionshistorie (list): Liste der Transaktionen.
    """

    def __init__(self, kontonummer, inhaber, bank_objekt, saldo=0):
        """
        Initialisiert ein neues Bankkonto.

        Args:
            kontonummer (str): Die Kontonummer.
            inhaber (Kundenklasse): Der Kontoinhaber.
            bank_objekt (Bankklasse): Die zugehörige Bank.
            saldo (float, optional): Startguthaben. Standard ist 0.
        """
        self.kontonummer = kontonummer 
        self.inhaber = inhaber
        self.bank = bank_objekt 
        self.iban = f"DE65 {self.bank.BLZ}{self.kontonummer}" # IBAN generieren
        self.pin = self.pin_generator()
        self.saldo = saldo
        self.dispo = 500.00
        self.transaktionshistorie = []

    @staticmethod
    def pin_generator():
        """
        Generiert eine zufällige 4-stellige PIN.

        Returns:
            str: Die generierte PIN.
        """
        neuer_pin = "".join([str(randint(0, 9)) for _ in range(4)])
        return neuer_pin
    
    def einzahlen(self, betrag):
        """
        Zahlt einen Betrag auf das Konto ein.

        Args:
            betrag (float): Der einzuzahlende Betrag.
        """
        self.saldo += betrag
        self.transaktionshistorie.append(("Einzahlung:" ,betrag,self.saldo))
        print(f"Neuer Kontostand nach Einzahlung: {self.saldo}")

    def abheben(self, betrag):
        """
        Hebt einen Betrag vom Konto ab, sofern ausreichend Guthaben vorhanden ist.

        Args:
            betrag (float): Der abzuhebende Betrag.
        """
        if betrag > self.saldo + self.dispo:
            print("Nicht genug Guthaben auf dem Konto")
        else:
            self.saldo -= betrag
            self.transaktionshistorie.append(("Abhebung:" ,betrag,self.saldo)) 
            print(f"Neuer Kontostand nach Abhebung: {self.saldo}")

    def konto_details(self):
        """
        Gibt die Kontodetails aus.
        """
        print("-" * 30)
        print(f"\nIBAN: {self.iban}\nInhaber: {self.inhaber}\nSaldo: {self.saldo}")
    
    def guthaben_details(self):
        """
        Gibt den aktuellen Saldo und Dispo aus.
        """
        print(f"Saldo: {self.saldo}\nDispo: {self.dispo}")   

    def Transaktionen(self):
        """
        Gibt die Transaktionshistorie des Kontos aus.
        """
        print("Transaktionshistorie:")
        for eintrag in self.transaktionshistorie:
            print(eintrag)


# Tagesgeldkonto vererbung von Bankkonto

class Tagesgeldkonto(Bankkonto):
    def __init__(self, kontoinhaber, kontonummer, betrag_limit):
        super().__init__(kontoinhaber, kontonummer,500)
        self.betrag_limit = betrag_limit

    def auszahlen(self, betrag):
        if betrag>self.betrag_limit:
            return False
        super().auszahlen(betrag)

In [3]:
class Tagesgeldkonto(Bankkontoklasse):
    def __init__(self, kontonummer, inhaber, bank_objekt, saldo=0, betrag_limit=1000.00):
        super().__init__(kontonummer, inhaber, bank_objekt, saldo, 1000.00)
        self.betrag_limit = betrag_limit

    def abheben(self, betrag):
        if betrag > self.betrag_limit:
            print(f"Abhebebetrag überschreitet das Limit von {self.betrag_limit}")
        else:
            super().abheben(betrag)  
         

# BankKlasse

In [4]:
import datetime
import random
#from Kundendatei import Kundenklasse as Kunde
#from Bankkontodatei import Bankkontoklasse as Bankkonto

class Bankklasse:
    def __init__(self, name, leiter, adresse, region):
        self.name = name
        self.leiter = leiter
        self.adresse = adresse
        self.region = region
        self.bankkonten = 0
        self.BLZ = "08051990"
        self.kunden_liste:list[Kundenklasse] = []
        self.konten_liste:list[Bankkontoklasse] = []

    def kontonummer_generieren(self):
        while True:
            kontonummer = "".join([str(random.randint(0, 9)) for _ in range(10)])
            if not any(konto.kontonummer == kontonummer for konto in self.konten_liste):
                print(f"Generierte Kontonummer: {kontonummer}")
                return kontonummer

    def konto_erstellen(self, inhaber, saldo = 0):
        kontonummer = self.kontonummer_generieren()
        neues_konto = Bankkontoklasse(kontonummer, inhaber, self, saldo)
        self.konten_liste.append(neues_konto)
        self.kunden_liste.append(inhaber)
        self.bankkonten += 1
        print(f"Neues Konto erstellt für {inhaber.name} mit Kontonummer {kontonummer}")
        return neues_konto
    
    def Überweissung(self, von_kontonummer, zu_kontonummer, betrag):
        konto_von = next((konto for konto in self.konten_liste 
                          if konto.kontonummer == von_kontonummer), 
                          None)
        konto_zu = next((konto for konto in self.konten_liste 
                         if konto.kontonummer == zu_kontonummer), 
                         None)
        
        if not konto_von:
            print(f"Konto mit Kontonummer {von_kontonummer} nicht gefunden.")
            return False
        if not konto_zu:
            print(f"Konto mit Kontonummer {zu_kontonummer} nicht gefunden.")
            return False
        if betrag > konto_von.saldo + konto_von.dispo:
            print("Nicht genug Guthaben für die Überweisung.")
            return False
        
        konto_von.abheben(betrag)
        konto_zu.einzahlen(betrag)
        print(f"Überweisung von {betrag} von Konto {von_kontonummer} zu Konto {zu_kontonummer} erfolgreich.")
        return True

    def konto_löschen(self, kontonummer):
        for konto in self.konten_liste:
            if konto.kontonummer == kontonummer:
                self.konten_liste.remove(konto)
                self.bankkonten -= 1
                print(f"Konto mit Kontonummer {kontonummer} wurde gelöscht.")
                return True
        print(f"Konto mit Kontonummer {kontonummer} nicht gefunden.")
        return False
    
    def bank_details(self):
        print(f"Bankname: {self.name}\nLeiter: {self.leiter}\nAdresse: {self.adresse}\nRegion: {self.region}\nAnzahl Konten: {self.bankkonten}")    
    
    


Verbleibende Aufgaben in die Bank Klasse:

1. Bank details ausgeben. Hier sollen keine Informationen ausgegeben werden. Nur quasi Werbung information bspw. Wie viele kontos hat die Bank, wie viele kunden hat die Bank, name der Bank, hauptstandort und land. bank_details() -> str. ca. 10 
2. kontos infos ausgeben. kunden infos ausgeben. Zwei unterschiedliche Methoden. ca. 30

3. Transaktion methode. Hier muss zuerst die Klasse Bankkonto erweitert werden mit eine Methode die folgende Signatur (erste Zeile einer Methode) hat.
   1. def(self,anderen_konto:bankkonto,betrag): -> True/False
4. Zweitens muesste fuer eine Transaktion eine Methode in Bank geschrieben werden, wo wir die Kontos mit einander verknuepfen wenn beide Kontos in self.kontos ueberhaupt existieren.
   1. def(self,von_konto,zu_konto,betrag): -> True/False

In [5]:
meine_bank = Bankklasse("Commerzbank", "CEO", "Zentrum 1", "DE-SH-SL/FL")

kunde1 = Kundenklasse("Anna", "Schmidt", "15.05.1990", "Beispielweg 5, 12345 Berlin", "0171-1234567", "anna.schmidt@email.com", "weiblich")

konto1 = meine_bank.konto_erstellen(inhaber=kunde1, saldo=500.00)

#meine_bank.bank_details()

Generierte Kundennummer: 2031538974
Generierte Kontonummer: 7249589347
Neues Konto erstellt für Anna Schmidt mit Kontonummer 7249589347


In [6]:
kunde2 = Kundenklasse("Jack", "Muster", "05.12.1990", "Musterweg 5, 12345 Köln", "0171-1234567", "Jack.Muster@email.com", "Mänlich")

konto2 = meine_bank.konto_erstellen(inhaber=kunde2, saldo=500.00)

Generierte Kundennummer: 4908671728
Generierte Kontonummer: 8603254634
Neues Konto erstellt für Jack Muster mit Kontonummer 8603254634


In [7]:
Bankkontoklasse.konto_details(konto1)
Bankkontoklasse.konto_details(konto2)


------------------------------

IBAN: DE65 080519907249589347
Inhaber: <__main__.Kundenklasse object at 0x000002536C01E030>
Saldo: 500.0
------------------------------

IBAN: DE65 080519908603254634
Inhaber: <__main__.Kundenklasse object at 0x000002536C01DD60>
Saldo: 500.0


In [8]:
Bankklasse.Überweissung(meine_bank, konto1.kontonummer, konto2.kontonummer, 200)

Neuer Kontostand nach Abhebung: 300.0
Neuer Kontostand nach Einzahlung: 700.0
Überweisung von 200 von Konto 7249589347 zu Konto 8603254634 erfolgreich.


True

## Bank Menue aufgabe:
* Start Menue definieren: Hier darf der Benutzer einen Kunden werden, sich einloggen oder das Bank System beenden.
  * Kunde anlegen: Nach anlegen wird der Kunde automatisch schon eingelogt. D.h. Zum naechsten Menue weitergeleitet.
  * Einloggen: anhand von vorname, nachname, adresse Kunde suchen. Wenn gefunden zum naechsten Menue weiterleiten. Ansonsten Wieder zum Start Menue zurueck.
  * System beenden.
* Nach Start Menue, haben wir ein Kunden Menue. Hier darf der Kunde seine persoenliche Daten ausgeben lassen oder auch einen Konto erstellen.
  * Details ausgeben
  * Bankkonto erstellen
  * Zurueck zum Start Menue.
* Nachdem ein Konto erstellt wird, dann kommt einen Konto Menue wo der Benutzer auswaehlen darf ob er einzahlen, auszahlen, ueberweisung, kontoauszug oder kontodetails sehen will.
  * einzahlen
  * auszahlen
  * ueberweisung
  * kontoauszug
  * kontodetails
  * Zurueck zum Kunde Menue
  * Wenn eine operation vorbei ist, dann kehren wir zurueck zu unser Konto Menue. Von hier duerfen wir weitere optionen auswaehlen.

In [14]:
#import NutzlicheMethoden as nm
from NutzlicheMethoden import telefonnummer_extrahieren as tele_ex

def start_menue():
    while True:
        print("\n---///--- Start Menü ---///---")
        print("      1. Kunde werden")
        print("      2. Einloggen")
        print("      3. System beenden")
        wahl = input("Bitte wählen Sie eine Option (1-3): ")

        if wahl == "1":
            print("\nOption 'Kunde werden' gewählt.")
            vorname=input("Vorname: ")
            nachname=input("Nachname: ")
            geburtsdatum=input("Geburtsdatum (TT.MM.JJJJ): ")
            adresse=input("Adresse: ")

            telefonnummer=input("Telefonnummer: ")

            telefonnummer = tele_ex(telefonnummer)

            email=input("E-Mail: ")
            geschlecht=input("Geschlecht: ")

            neuer_kunde = Kundenklasse(
                vorname,nachname,geburtsdatum,adresse,telefonnummer,email,geschlecht
            )
            # Wichtig: Die Kontodaten werden in einer separaten Variable gespeichert.
            neues_konto = meine_bank.konto_erstellen(inhaber=neuer_kunde, saldo=0.00)
            print(neues_konto.pin)
            kunden_menue(neuer_kunde, neues_konto)
            
        elif wahl == "2":
            print("\nOption 'Einloggen' gewählt.")
            kundennummer = input("Bitte geben Sie Ihre Kundennummer ein: ")
            
            gefundener_kunde = None
            for kunde in meine_bank.kunden_liste:
                if kunde.kundennummer == kundennummer:
                    gefundener_kunde = kunde
                    break
            
            if gefundener_kunde:
                print(f"Willkommen zurück, {gefundener_kunde.name}!")
                enter_pin = input("Bitte geben Sie Ihre PIN ein: ")
                
                konto = None
                for k in meine_bank.konten_liste:
                    if k.inhaber.kundennummer == kundennummer:
                        konto = k
                        break
                
                if konto and enter_pin == konto.pin:
                    print("Erfolgreich eingeloggt!")
                    kunden_menue(gefundener_kunde, konto)
                else:
                    print("Falsche PIN oder kein Konto gefunden. Bitte versuchen Sie es erneut.")
            else:
                print("Kundennummer nicht gefunden. Bitte versuchen Sie es erneut.")
                
        elif wahl == "3":
            print("\nDas Banksystem wird beendet.")
            break
        else:
            print("\nUngültige Eingabe, bitte versuchen Sie es erneut.")


In [15]:
def kunden_menue(kunde, konto):
    while True:
        print("\n---///--- Willkommen im Kunden Menü. ---///---")
        print("         1. Kundendetails anzeigen")
        print("         2. Konto erstellen")
        print("         3. Konto löschen")
        print("         4. Konto Menü")
        print("         5. Abmelden")
        wahl = input("Bitte wählen Sie eine Option (1-5): ")

        if wahl == "1":
            print("\nOption 'Kundendetails anzeigen' gewählt.")
            kunde.Kundendetails()
        
        elif wahl == "2":
            print("\nOption 'Konto erstellen' gewählt.")
            # Es wird ein neues Konto erstellt. Da ein Kunde mehrere Konten haben kann,
            # wird es nicht automatisch als das aktuelle Konto gesetzt.
            neues_konto = meine_bank.konto_erstellen(inhaber=kunde, saldo=0.00)
            print(f"Ein neues Konto mit der Nummer {neues_konto.kontonummer} wurde erstellt!")
        
        elif wahl == "3":
            print("\nOption 'Konto löschen' gewählt.")
            kontonummer = input("Bitte geben Sie die Kontonummer ein, die Sie löschen möchten: ")
            meine_bank.konto_löschen(kontonummer)

        elif wahl == "4":
            print("\nOption 'Konto Menü' gewählt.")
            option = konto_menue(konto)
            if option == -1:
                return
            else:
                continue

        elif wahl == "5":
            print("\nSie werden abgemeldet.")
            break
        else:
            print("\nUngültige Eingabe, bitte versuchen Sie es erneut.")


In [16]:
def konto_menue(konto):
    while True:
        print("\n---///--- Wilkommen im Konto Menü. ---///---")
        print("                 1. Auszahlung")
        print("                 2. Einzahlen")
        print("                 3. Überweisung")
        print("                 4. Kontoauszug")
        print("                 5. Kontodetails")
        print("                 6. Kunden Menü")
        print("                 7. Abmelden")
        wahl = input("Bitte wählen Sie eine Option (1-8): ")
        if wahl == "1":
            print("\nOption 'Auszahlung' gewählt.")
            betrag = float(input("Geben Sie den Betrag ein, den Sie auszahlen möchten: "))
            konto.abheben(betrag)
        elif wahl == "2":
            print("\nOption 'Einzahlen' gewählt.")
            betrag = float(input("Geben Sie den Betrag ein, den Sie einzahlen möchten: "))
            konto.einzahlen(betrag)
        elif wahl == "3":
            print("\nOption 'Überweisung' gewählt.")
            zielkontonummer = input("Geben Sie die Zielkontonummer ein: ")
            betrag = float(input("Geben Sie den Betrag ein, den Sie überweisen möchten: "))
            meine_bank.Überweissung(konto.kontonummer, zielkontonummer, betrag)
        elif wahl == "4":
            print("\nOption 'Kontoauszug' gewählt.")
            konto.Transaktionen()
        elif wahl == "5":
            print("\nOption 'Kontodetails' gewählt.")
            konto.konto_details()
        elif wahl == "6":
            print("\nSie werden zum Kunden Menü zurückgeleitet.")
            return 1
        elif wahl == "7":
            print("\nSie werden abgemeldet und zum Start Menü zurückgeleitet.")
            return -1
        else:
            print("\nUngültige Eingabe, bitte versuchen Sie es erneut.")

#Start->Kunden->Konto

In [17]:
print("--- Kundenliste ---")
for kunde in meine_bank.kunden_liste:
    print(f"Name: {kunde.name}, Kundennummer: {kunde.kundennummer}")

print(f"\nDer Pin von Konto 1 ist: {konto1.pin}")
print(f"Der Pin von Konto 2 ist: {konto2.pin}")

print("\n--- Kontenliste ---")
for konto in meine_bank.konten_liste:
    print(f"Kontonummer: {konto.kontonummer}, Inhaber: {konto.inhaber.name}, Saldo: {konto.saldo}")

--- Kundenliste ---
Name: Anna Schmidt, Kundennummer: 2031538974
Name: Jack Muster, Kundennummer: 4908671728

Der Pin von Konto 1 ist: 9177
Der Pin von Konto 2 ist: 2477

--- Kontenliste ---
Kontonummer: 7249589347, Inhaber: Anna Schmidt, Saldo: 300.0
Kontonummer: 8603254634, Inhaber: Jack Muster, Saldo: 700.0


In [18]:
# Starte das Menü
start_menue()


---///--- Start Menü ---///---
      1. Kunde werden
      2. Einloggen
      3. System beenden

Option 'Kunde werden' gewählt.
Generierte Kundennummer: 1364602432
Generierte Kontonummer: 7047874077
Neues Konto erstellt für Nicolas Arevalo mit Kontonummer 7047874077
4345

---///--- Willkommen im Kunden Menü. ---///---
         1. Kundendetails anzeigen
         2. Konto erstellen
         3. Konto löschen
         4. Konto Menü
         5. Abmelden

Option 'Kundendetails anzeigen' gewählt.
Name: Nicolas Arevalo
Alter 15
Adresse: Musterstrasse 1
Telefonnummer: 2347645376
E-Mail: muster@outlook.de
Geschlecht: M

---///--- Willkommen im Kunden Menü. ---///---
         1. Kundendetails anzeigen
         2. Konto erstellen
         3. Konto löschen
         4. Konto Menü
         5. Abmelden

Ungültige Eingabe, bitte versuchen Sie es erneut.

---///--- Willkommen im Kunden Menü. ---///---
         1. Kundendetails anzeigen
         2. Konto erstellen
         3. Konto löschen
         4. K